In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
COCOAPIROOT = r"D:\学习资料\实验室"
from pycocotools.coco import COCO
from raw_program.data_loader import get_loader
from raw_program.models import EncoderCNN, DecoderWithAttention
import math


# 选取合适参数
batch_size = 1          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity
attention_dim = 512
embed_dim = embed_size
decoder_dim = 512 

# 建立transforms
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# 建立dataloader
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file,
                         cocoapi_loc=COCOAPIROOT)

loading annotations into memory...
Done (t=1.14s)
creating index...
index created!
[0/591753] 正在读取captions并根据其分词建立词典...
[100000/591753] 正在读取captions并根据其分词建立词典...
[200000/591753] 正在读取captions并根据其分词建立词典...
[300000/591753] 正在读取captions并根据其分词建立词典...
[400000/591753] 正在读取captions并根据其分词建立词典...
[500000/591753] 正在读取captions并根据其分词建立词典...
初始化vocab.pkl文件成功
loading annotations into memory...
Done (t=1.11s)
creating index...
index created!
正在对caption分词...


100%|████████████████████████████████████████████████████████████████████████| 591753/591753 [01:01<00:00, 9637.36it/s]


In [2]:
# 定义词典大小
vocab_size = len(data_loader.dataset.vocab)

# 初始化encoder和decoder 
encoder = EncoderCNN(embed_size)
decoder = DecoderWithAttention(attention_dim, embed_dim, decoder_dim, vocab_size)

# 把模型移动到GPU中
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# 定义损失函数
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# 把可学习的参数建立一个列表
params = list(decoder.parameters())

# 选定优化器
optimizer = torch.optim.Adam(params, lr=0.001)

# 设置每一个epoch训练多少步
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

D:\python\anaconda\envs\PyTorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
D:\python\anaconda\envs\PyTorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Check model save/load
import os

# Save
torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-0.pkl'))
torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-0.pkl'))
# Load
decoder_file = 'decoder-0.pkl'
encoder_file = 'encoder-0.pkl'
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file),  map_location='cpu'))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file),  map_location='cpu'))

<All keys matched successfully>

In [4]:
import torch.utils.data as data
import numpy as np

import requests
import time
import torch, gc


# Open the training log file.
f = open(log_file, 'w')
# Select True if training on local desktop. False to train on GPU workspace
local = False
# if not local:
start_time = time.time()
#     response = requests.request("GET", 
#                                 "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                                 headers={"Metadata-Flavor":"Google"})
for epoch in range(1, num_epochs+1):
    for i_step in range(1, total_step+1):
#         if not local:
#             if time.time() - old_time > 60:
#                 old_time = time.time()
#                 requests.request("POST", 
#                                  "https://nebula.udacity.com/api/v1/remote/keep-alive", 
#                                  headers={'Authorization': "STAR " + response.text})
        # 随机从caption_length中采样返回对应索引
        indices = data_loader.dataset.get_train_indices()
        # 创建一个样本
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        # 获取这批数据
        images, captions = next(iter(data_loader))
        # 将数据移到GPU中
        images = images.to(device)
        captions = captions.to(device)
        caption_lengths_2 = torch.full((batch_size, 1), len(captions[0]))
        caption_lengths_2 = caption_lengths_2.to(device)
        # 将梯度归零
        decoder.zero_grad()
        encoder.zero_grad()
        
       
        gc.collect()
        torch.cuda.empty_cache()
        # 将输入传到encoder和decoder中
        with torch.no_grad():
            features = encoder(images)
            outputs = decoder(features, captions, caption_lengths_2)[0]
        
        # 计算损失函数
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        # 反向传播
        loss.backward()
        # 更新优化器参数
        optimizer.step()
        # 获取训练的统计数据
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        # 打印训练数据(同一行中)
        print('\r' + stats, end="")
        sys.stdout.flush()
        # 将数据存到文件中
        f.write(stats + '\n')
        f.flush()
        # 打印训练数据(换行)
        if i_step % print_every == 0:
            print('\r' + stats)

    # 保存权重
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# 关闭log文件
f.close()
end_time = time.time()
print("本次训练时长：{}".format(end_time-start_time))

RuntimeError: CUDA out of memory. Tried to allocate 2.00 GiB (GPU 0; 6.00 GiB total capacity; 2.17 GiB already allocated; 1.96 GiB free; 2.20 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
len(data_loader.dataset.caption_lengths)

In [ ]:
caption_lengths_2

In [ ]:
captions